In [2]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import time
current_record = 0

In [7]:
with open("WP2021.txt", 'r') as f:
    cases = f.read()
    year = 2019


csv_file = open('2019records.csv','a', encoding='utf-8', newline='')
csvwriter = csv.writer(csv_file)
# if current_record == 0:
#     fields = ['P1', "P2", "P3", "P4", "P_Total", 'P_Advocate', 'R1',"R2", "R3", "R4","R_Total", 'filing_number', 'filing_year', 'filing_date', 'registration_number', 'registration_year', 'registration_date','cnr', 'judge', 'bench','statute', 'provision', 'first_hearing', 'stage', 'category', 'sub_category', 'sub_sub_category', 'last_hearing_date']
#     csvwriter.writerow(fields) 

cases = cases.split("##")
for x in range(current_record, len(cases)):
    prep_stage = cases[x].split("~")
    cino = prep_stage[3]
    case_no = prep_stage[0]
    #print(current_record, cino, case_no)
    try:
        soup = BeautifulSoup(get_case_deets(cino, case_no))
    except:
        time.sleep(0.5)
        continue
    try:
        row = process(soup)
        print_all(soup)
        #print(row)
        #print()
    except Exception as e:
        print("ERROR", current_record)
    if row != 0:
        csvwriter.writerow(row)
    current_record+=1

    with open(f"case_files/{year}_{current_record}.html", 'w', encoding="utf-8") as f:
        f.write(str(soup))



ERROR 1544
ERROR 2843
ERROR 5257
ERROR 5586
ERROR 6316
ERROR 11442
ERROR 11855


In [227]:
csv_file.close()

In [3]:
def get_case_deets(cino, case_no):
    s = requests.Session()
    s.get("https://services.ecourts.gov.in/ecourtindiaHC/cases/s_casetype.php?state_cd=3&dist_cd=1&court_code=1&stateNm=Karnataka#")


    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="97", "Chromium";v="97"',
        'DNT': '1',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
        'sec-ch-ua-platform': '"Windows"',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': '*/*',
        'Origin': 'https://services.ecourts.gov.in',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://services.ecourts.gov.in/',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    data = {
      '__csrf_magic': 'sid:22e25a4a3b8a2c3dd3a08f8c43e207354a236ed2,1641670946',
      'state_code': '3',
      'dist_code': '1',
      'cino': cino,
      'case_no': case_no,
      'appFlag': '',
      'court_code': '1'
    }

    response = s.post('https://services.ecourts.gov.in/ecourtindiaHC/cases/o_civil_case_history.php', headers=headers,data=data)
    
    return response.text

In [4]:

def process(soup):
    deets = get_all(soup)
    if len(deets[1])<1:
        return 0
    disposed_date = " "
    disposed_reason = " "
    filing_number= deets[0][6][15:25].split('/')[0]
    filing_date = deets[0][8][-10:]
    filing_year = filing_date[-4:]
    registration_number = deets[0][9][21:31].split('/')[0]
    registration_date = deets[0][11][19:]
    registration_year = registration_date[-4:]
    stage = deets[0][16].split(": ")[-1].strip()       
    
    if stage=='CASE DISPOSED':
        disposed_date = deets[0][15].split(":")[-1].strip()
        disposed_reason =deets[0][17].split(":")[-1].strip()
        
        judge = deets[0][18].split(':')[-1].strip()[4:].title()
        bench = deets[0][19].split(':')[-1].strip().title()
    else:
        judge = deets[0][17].split(':')[-1].strip()[4:].title()
        bench = deets[0][18].split(':')[-1].strip().title()
    if bench == '':
        bench = ' '
    cnr = cino
    statute = deets[1][0]
    provision = deets[1][1]
    if statute == 'History of Case Hearing':
        statute = " "
        provision = " "
    
    try:
        first_hearing = deets[0][14].split(": ")[1]
    except IndexError:
        first_hearing = ""

    category, sub_category, sub_sub_category, *_ = get_category_details(deets[-1])

    p_advocate, p_length, p1, p2, p3, p4, *_ = clean_parties(re.split(r'\d\)',deets[0][-2]))
    r_advocate, r_length, r1, r2, r3, r4, *_ = clean_parties(re.split(r'\d\)',deets[0][-1]))
    

    try:
        last_hearing = soup.select("body > form > div:nth-child(2) > table.history_table > tr > td")
        last_hearing_date = last_hearing[-3].get_text()
        last_hearing_matter = last_hearing[-1].get_text()
    except IndexError:
        last_hearing_date = ""
        last_hearing_matter = ""
    
    try:
        order_link = soup.select("#secondpage > div:nth-child(13) > table.order_table > tbody > tr:nth-child(2) > td:nth-child(4) > a")['href']
    except:
        order_link = " "
    rows = [p1,p2,p3,p4,p_length, p_advocate, r1,r2,r3,r4,r_length, filing_number, filing_year, filing_date, registration_number, registration_year, registration_date, cnr, judge, bench, statute, provision, first_hearing, stage, category, sub_category, sub_sub_category, last_hearing_date, disposed_date, disposed_reason, order_link]

    return rows


In [5]:
def print_all(soup):
    import unicodedata
    f = open('log.txt', 'a')
    normal = lambda x: unicodedata.normalize("NFKD", x.get_text().strip())
    f.write(f"Record: {current_record}: ")
    f.write(str([normal(x) for x in soup.find_all('span')]))
    f.write('td\n')
    f.write(str([normal(x) for x in soup.find_all('td')]))
    f.write('tr\n')
    f.write(str([normal(x) for x in soup.find_all('tr')]))
    f.write('\n')
    last_hearing = soup.select("body > form > div:nth-child(2) > table.history_table > tr > td")
    f.write(str(last_hearing))
    f.write('\n\n')

    f.close()
def get_all(soup):
    import unicodedata
    normal = lambda x: unicodedata.normalize("NFKD", x.get_text().strip())
    return (([normal(x) for x in soup.find_all('span')]), [normal(x) for x in soup.find_all('td')], [normal(x) for x in soup.find_all('tr')])\

def get_category_details(items):
    temp_category = [x.split('\n')[-1] for x in items if "Category\n" in x]
    while len(temp_category)<3:
        temp_category.append(" ")
    return temp_category

def clean_parties(items):
    parties = []
    advocate = ''
    for x in items:
        if len(x)<2:
            continue
        advocate = x.split("Advocate-")[-1].strip()
        if type(advocate)!=str:
            advocate = " "
        x = x.split("Advocate")[0].strip()
        if x[-1] == ',':
            parties.append(x[:-1].title())
        else:
            parties.append(x.title())
    parties.insert(0, len(parties))
    parties.insert(0, advocate)
    while len(parties)<6:
        parties.append(" ")
    return parties

In [143]:
print([].append(""))

None


In [85]:

last_hearing = soup.select("body > form > div:nth-child(2) > table.history_table > tr > td")

last_hearing_date = last_hearing[-3].get_text()
last_hearing_matter = last_hearing[-1].get_text()

In [2]:
import pandas as pd

df = pd.read_csv("records.csv")

In [9]:
df.columns

Index(['P1', 'P2', 'P3', 'P4', 'P_Total', 'P_Advocate', 'R1', 'R2', 'R3', 'R4',
       'R_Total', 'Filing Number', 'filing_year', 'filing_date',
       'registration_number', 'registration_year', 'registration_date', 'cnr',
       'judge', 'bench', 'statute', 'provision', 'first_hearing', 'stage',
       'category', 'sub_category', 'sub_sub_category', 'last_hearing_date',
       'date_dismissed', 'dismissal_type'],
      dtype='object')

In [14]:
df.groupby(['sub_category', 'sub_sub_category'])['P1'].count()

sub_category                                       sub_sub_category                               
                                                                                                      144
APMC-Agrl.Produce (Marketing)Regulation Act ( 2 )                                                      39
BDA-Bangalore Development Authority Act ( 3 )                                                         199
                                                   PIL-Public Interest Litigation ( 1 )                 3
C-Cinema Regulation Act ( 4 )                                                                           3
                                                                                                     ... 
T-Tax Matter ( 21 )                                RES-Residuary ( 112 )                              241
                                                   TAR-Central Excise/Customs Act (Tariff) ( 123 )      2
T-Tax Matter ( 32 )                                IT